In [1]:
import xarray as xr
from paths import aq_directory_local
from meteorology import *
import numpy as np
import plotly.graph_objects as go

from astropy.stats import circmean
from astropy import units as u
from meteorology import get_factor_data

In [2]:
# file = aq_directory_local + 'meteoData_2019.nc'
file = '../Files/meteoData_2019.nc'

meteo_data = xr.open_dataset(file)['meteo']
meteo_data

<xarray.DataArray 'meteo' (district: 30, time: 8760, factor: 37)>
[9723600 values with dtype=float64]
Coordinates:
  * time      (time) datetime64[ns] 2019-01-01 ... 2019-12-31T23:00:00
  * factor    (factor) object 'Temperature [2 m elevation corrected]' ... 'Evapotranspiration'
  * district  (district) object 'Barisal' 'Bogra' ... 'Tongi' 'Tungipara'

In [3]:
meteo_data.time

<xarray.DataArray 'time' (time: 8760)>
array(['2019-01-01T00:00:00.000000000', '2019-01-01T01:00:00.000000000',
       '2019-01-01T02:00:00.000000000', ..., '2019-12-31T21:00:00.000000000',
       '2019-12-31T22:00:00.000000000', '2019-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2019-01-01 ... 2019-12-31T23:00:00

# Analysis of wind direction

In [11]:
# meteo_data_wind_direction = meteo_data.loc[:, :, wind_direction_factors()[0]]
# for month,data in meteo_data_wind_direction.groupby('time.month'):
#     # print(data)
#     print(data.stack(z=("time", "district","factor")).values)

In [5]:

wind_direction_10m = get_factor_data(meteo_data,'Wind Direction [10 m]')
wind_direction_100m = get_factor_data(meteo_data,'Wind Direction [100 m]')
wind_direction_250m = get_factor_data(meteo_data,'Wind Direction [250 mb]')
wind_direction_500m = get_factor_data(meteo_data,'Wind Direction [500 mb]')
wind_direction_700m = get_factor_data(meteo_data,'Wind Direction [700 mb]')
wind_direction_850m = get_factor_data(meteo_data,'Wind Direction [850 mb]')
wind_direction_900m = get_factor_data(meteo_data,'Wind Direction [900 mb]')

In [ ]:
wind_direction_10m

In [ ]:
wind_direction_700m

In [15]:
# wind_dir_10m_bd = wind_direction_10m.stack().droplevel(1)
# wind_dir_100m_bd = wind_direction_100m.stack().droplevel(1)
# wind_dir_10m_bd

### Linear statistics won't work for circular data like wind direction, so we will use circular statistics

In [8]:
def circular_mean(data):
    # data = np.array([51, 67, 40, 109, 31, 358])
    # data = np.array([2, 3, 40, 339, 352, 358])
    return (circmean(data*u.deg)/u.deg+360)%360

data = np.array([1,358])
circular_mean(data)

<Quantity 359.5>

In [9]:
monthly_direction_10m = wind_direction_10m.resample('M').apply(lambda x: circular_mean(x.values))
monthly_direction_100m = wind_direction_100m.resample('M').apply(lambda x: circular_mean(x.values))
monthly_direction_250m = wind_direction_250m.resample('M').apply(lambda x: circular_mean(x.values))
monthly_direction_500m = wind_direction_500m.resample('M').apply(lambda x: circular_mean(x.values))
monthly_direction_700m = wind_direction_700m.resample('M').apply(lambda x: circular_mean(x.values))
monthly_direction_850m = wind_direction_850m.resample('M').apply(lambda x: circular_mean(x.values))
monthly_direction_900m = wind_direction_900m.resample('M').apply(lambda x: circular_mean(x.values))

In [10]:
monthly_direction_10m

time
2019-01-31     326.4141048941575
2019-02-28    311.06192609062043
2019-03-31    234.36424105966233
2019-04-30     141.5042698526571
2019-05-31    168.27150003685824
2019-06-30    150.81651817905924
2019-07-31    155.89573116678673
2019-08-31     137.3709370030033
2019-09-30    135.75442706500803
2019-10-31    342.74523025995785
2019-11-30    340.74677901813607
2019-12-31    341.15911771114787
dtype: object

In [15]:
monthly_direction_700m

time
2019-01-31    271.24242037083354
2019-02-28    274.06197099752717
2019-03-31    283.90079570286787
2019-04-30     281.4175006696936
2019-05-31      281.255189469533
2019-06-30      276.483358552905
2019-07-31    201.60995310008644
2019-08-31     135.3695411693957
2019-09-30     177.2240702237383
2019-10-31     249.1590757179073
2019-11-30    279.99312355946904
2019-12-31     279.2406250362859
dtype: object

### We can observe that wind direction with 10 meter height is consistent with seasonal wind direction of Bangladesh but not so much for height of 700 meter

In [23]:
monthly_direction_10m.astype(int).corr(monthly_direction_900m.astype(int))
# monthly_direction_10m.astype(int).corr(monthly_direction_500m.astype(int))
# monthly_direction_500m.astype(int).corr(monthly_direction_700m.astype(int))

0.9186367218469381

In [21]:
# import seaborn as sns
#
# sns.set()
# sns.pointplot(data=monthly_direction_10m)

# Analysis of wind speed

In [28]:
wind_speed_10m = get_factor_data(meteo_data,'Wind Speed [10 m]')
wind_speed_100m = get_factor_data(meteo_data,'Wind Speed [100 m]')
wind_speed_250m = get_factor_data(meteo_data,'Wind Speed [250 mb]')
wind_speed_500m = get_factor_data(meteo_data,'Wind Speed [500 mb]')
wind_speed_700m = get_factor_data(meteo_data,'Wind Speed [700 mb]')
wind_speed_850m = get_factor_data(meteo_data,'Wind Speed [850 mb]')
wind_speed_900m = get_factor_data(meteo_data,'Wind Speed [900 mb]')

In [23]:
wind_speed_10m.stack().describe()

count    262800.000000
mean          8.509030
std           4.393377
min           0.000000
25%           5.506941
50%           7.695920
75%          10.703569
max          45.950440
dtype: float64

In [24]:
wind_speed_700m.stack().describe()

count    262800.000000
mean         33.256230
std          18.103324
min           0.000000
25%          19.376562
50%          30.955812
75%          44.250720
max         114.408090
dtype: float64

In [67]:
(wind_speed_500m.stack()<15).sum()/262800.000000

0.13435312024353122

In [ ]:
fig = go.Figure()
meteo_data_wind_speed = meteo_data.loc[['Dhaka'], :, wind_speed_factor.factor_list[:2]]
for month,data in meteo_data_wind_speed.groupby('time.month'):
    fig.add_trace(go.Box(y=data.stack(z=("time", "district","factor")).values, name=str(month), marker_color=wind_speed_factor.color_list[6]))
fig.show()

In [35]:
wind_speed_10m

time                 district 
2019-01-01 00:00:00  Barisal      7.787991
                     Bogra        8.145870
                     Dhaka        6.915374
                     Jamalpur     6.763786
                     Jessore      8.587338
                                    ...   
2019-12-31 23:00:00  Sherpur      6.696387
                     Sirajganj    4.024922
                     Tangail      2.811690
                     Tongi        8.311245
                     Tungipara    8.287822
Length: 262800, dtype: float64

In [ ]:
import plotly.figure_factory as ff
import numpy as np

# Add histogram data
x1 = wind_speed_10m.stack().sample(2628)
x2 = wind_speed_100m.stack().sample(2628)
x3 = wind_speed_250m.stack().sample(2628)
x4 = wind_speed_500m.stack().sample(2628)
x5 = wind_speed_700m.stack().sample(2628)
x6 = wind_speed_850m.stack().sample(2628)
x7 = wind_speed_900m.stack().sample(2628)

# Group data together
hist_data = [x1, x2, x3, x4, x5, x6, x7]

group_labels = ['Group 10', 'Group 100', 'Group 250', 'Group 500', 'Group 700', 'Group 850', 'Group 900']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.5,colors=wind_speed_factor.color_list)
fig.show()

In [ ]:

x0 = wind_speed_10m.stack().sample(262).values
# Add 1 to shift the mean of the Gaussian distribution
x1 = wind_speed_100m.stack().sample(262).values

# x0 = np.rand
# om.randn(500) + 1

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0))
fig.add_trace(go.Histogram(x=x1))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()